<h1>The BurnMan Tutorial</h1>
<h2>Part 5: Equilibrium problems</h2>
This file is part of BurnMan - a thermoelastic and thermodynamic toolkit
for the Earth and Planetary Sciences

Copyright (C) 2012 - 2021 by the BurnMan team,
released under the GNU GPL v2 or later.


# Introduction

This ipython notebook is the second in a series designed to introduce new users to the code structure and functionalities present in BurnMan.

## Demonstrates

1. burnman.equilibrate


Everything in BurnMan and in this tutorial is defined in SI units. 

## Importing BurnMan

In the first part of this tutorial, we decided whether to install the version of BurnMan associated with the tutorial or use the local version. If you chose to install globally, or if you want to use a previously installed version, change the "use_installed" variable below to "True".

In [ ]:
use_installed = False

if not use_installed:
    import os
    import sys
    if not os.path.exists('burnman') and os.path.exists('../../burnman'):
        sys.path.insert(1, os.path.abspath('../../'))

import burnman
print(f'This tutorial will use BurnMan version {burnman.__version__}.')

# Phase equilibria

## Multiphase equilibria the handrolled way
### The olivine polymorphs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root

def equilibrate_fe_mg(polymorphs, pressure, temperature):

  for p in polymorphs:
    p.set_state(pressure, temperature)

  def delta_mu(args):
    x_fe_p1, x_fe_p2 = args
    polymorphs[0].set_composition([1. - x_fe_p1, x_fe_p1])
    polymorphs[1].set_composition([1. - x_fe_p2, x_fe_p2])

    return polymorphs[0].partial_gibbs - polymorphs[1].partial_gibbs

  
  sol = root(delta_mu, [0.2, 0.4], method='hybr')
  if sol.success:
    for i in range(2):
      polymorphs[i].set_composition([1. - sol.x[i], sol.x[i]])
      polymorphs[i].set_state(pressure, temperature)

In [ ]:
from burnman import minerals

ol = minerals.SLB_2011.mg_fe_olivine()
wad = minerals.SLB_2011.mg_fe_wadsleyite()

pressures = np.linspace(12.e9, 13.75e9, 11)
x_fe_ol = np.empty_like(pressures)
x_fe_wad = np.empty_like(pressures)

for i, P in enumerate(pressures):
  equilibrate_fe_mg([ol, wad], P, 1600.)

  x_fe_ol[i] = ol.molar_fractions[1]
  x_fe_wad[i] = wad.molar_fractions[1]

plt.plot(x_fe_ol, pressures/1.e9, label='ol')
plt.plot(x_fe_wad, pressures/1.e9, label='wad')

plt.xlabel('p(Fe$_2$SiO$_4$)')
plt.ylabel('Pressure (GPa)')
plt.legend()
plt.show()

## Multiphase equilibria the efficient way
### The olivine polymorphs revisited

In the previous section, we created a custom function to calculate the equilibrium state between two phases. This is a tedious procedure, and one which users may wish to avoid! BurnMan now contains an experimental solver which can solve problems that can be expressed as an isochemical equilibrium problem.

The binary loop shown in the previous section can be constructed by choosing a series of compositions at 1600 K and asking BurnMan where wadsleyite first becomes stable (has a phase fraction of zero). Actually, we can do even more, and ask where wadsleyite makes up 0, 25, 50, 75 and 100\% of the rock.

In [ ]:
from burnman import equilibrate

assemblage = burnman.Composite([ol, wad])

n_compositions = 20
n_wad_fractions = 5
x_fe_ol = np.linspace(0., 1., n_compositions)
wad_fractions = np.linspace(0., 1., n_wad_fractions)
pressures = np.empty((20, 5))
for i, x_fa in enumerate(x_fe_ol):
  composition = {'Mg': 1. - x_fa, 'Fe': x_fa, 'Si': 0.5, 'O': 2.}
  equality_constraints = [['T', 1600.],
                          ['phase_fraction', [wad, wad_fractions]]]

  sols = burnman.equilibrate(composition, assemblage, equality_constraints)
  pressures[i] = [sol.assemblage.pressure for sol in sols[0]]

for i in range(n_wad_fractions):
  plt.plot(x_fe_ol, pressures[:,i]/1.e9, label=f'$f_{{wad}}$ = {wad_fractions[i]:.2f}')

plt.xlabel('p(Fe$_2$SiO$_4$)')
plt.ylabel('Pressure (GPa)')
plt.legend()
plt.show()